### Calculate fractional coverage (weights) for pixel intersecting a polygon layer
Inputs:
- Polygon layer
- Grid (ie, netcdf file) -- Based on NWM v3.0 retrospective forcing data
Output:
- Writes to weights table in the warehouse
- Table name: "grid_pixel_coverage_weights"
- Schema:
  - `fraction_covered`: double
  - `position_index`: int
  - `location_id`: string (this represents the polygon layer)
  - `configuration_name`: string (represents the model producing the gridded data)
  - `variable_name`: string (represents the actual grid)
  - `row`: int
  - `col`: int

In [ ]:
import geopandas as gpd
from pyspark.sql import functions as F
import botocore.session
import os

import teehr
from teehr.evaluation.spark_session_utils import create_spark_session

In [ ]:
CONFIGURATION_NAME = "nwm30_retrospective"
VARIABLE_NAME = "rainfall_hourly_rate"
TABLE_NAME = "grid_pixel_coverage_weights"

In [ ]:
%%time
dir_path = "/data/playground/slamont/teehr/warehouse/sedona/usgs_basins_map"  

spark = create_spark_session()

# USE EXISTING:
ev = teehr.Evaluation(
    spark=spark,
    dir_path=dir_path,
    create_dir=False
)

In [ ]:
ev.list_tables()

In [ ]:
ev.spark.sql("DESCRIBE local.teehr.nwm30_usgs_hires_basins_fractional_coverage").show()

### Read in the polygons for weights calculation.

We need to reproject the polygon layer to match the NWM v3.0 retro CRS. The proj4 string below was extracted from the Zarr attributes at "s3://noaa-nwm-retrospective-3-0-pds/CONUS/zarr/forcing/precip.zarr"

In [ ]:
nmw30_proj4 = "+proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1=30.0 +lat_2=60.0 +lat_0=40.0 +lon_0=-97.0 +x_0=0 +y_0=0 +k_0=1.0 +nadgrids=@null +wktext  +no_defs"

In [ ]:
gdf = gpd.read_parquet("/data/playground/slamont/usgs_basins/usgsbasin_geometry_highres.conus.parquet")
gdf_nwm = gdf.to_crs(crs=nmw30_proj4)
gdf_nwm["geometry"] = gdf_nwm.geometry.make_valid()  # NOTE: Had some invalid geometries. Always include this.
gdf_nwm.to_parquet("/data/playground/slamont/teehr/warehouse/sedona/usgsbasin_geometry_highres.conus.nwmCRS.parquet")  # Save the reprojected and validated version to disk.

Load the polygons into a spark dataframe

In [ ]:
poly_sdf = (
    spark
    .read
    .format("parquet")
    .load("/data/playground/slamont/teehr/warehouse/sedona/usgs_basin_geometry/usgsbasin_geometry_highres.conus.nwmCRS.parquet")
    .selectExpr("ST_GeomFromWKB(geometry) as geometry", "id")
)

### Read in a single grid file as a template

In [ ]:
filepath = "/data/playground/slamont/teehr/warehouse/sedona/197902010000.LDASIN_DOMAIN1"  # read in a local file.

raster_sdf = spark.read.format("binaryFile").load(filepath).selectExpr("RS_FromNetCDF(content, 'RAINRATE', 'x', 'y') as raster", "path as filepath") 

In [ ]:
raster_geom_sdf = raster_sdf.limit(1).selectExpr(
  "explode(RS_PixelAsPolygons(raster, 1)) as exploded"
).selectExpr(
  "exploded.geom as geom",
  "exploded.value as value",
  "exploded.x as col",
  "exploded.y as row"
)
raster_geom_sdf.show(3)

Calculate a 1-D "position index" that maps the row/col position using the grid dimensions (width)

In [ ]:
%%time
raster_width = raster_sdf.selectExpr("RS_Width(raster) as width", "RS_Height(raster) as height").collect()[0]["width"]

raster_geom_sdf = raster_geom_sdf.withColumn("position_index", ((F.col("row") - 1) * f"{raster_width}" + (F.col("col") - 1)))
raster_geom_sdf.show(3)

### Now we can calculate the weights and write to the warehouse

Note: These cells are for calculating a new table. Since the NWM 3.0/USGS Basins table was pre-calculated and saved locally, it was loaded below, skipping these cells. 

In [ ]:
# Not sure if this is necessary.
# This does not add partitions for unique values, but rather some default number unless specified (17?)
raster_geom_sdf = raster_geom_sdf.repartition("geom")
poly_sdf = poly_sdf.repartition("id")

In [ ]:
poly_sdf.createOrReplaceTempView("polygon_view")
raster_geom_sdf.createOrReplaceTempView("raster_polygons_view")

In [ ]:
weights_results_sdf = spark.sql("""
    SELECT 
        ST_Area(ST_Intersection(rpv.geom, pv.geometry)) / ST_Area(rpv.geom) as fraction_covered, rpv.position_index, rpv.row, rpv.col, pv.id as location_id    
    FROM raster_polygons_view AS rpv
    INNER JOIN polygon_view AS pv
    ON ST_Intersects(rpv.geom, pv.geometry)
""")

In [ ]:
coverage_weights_sdf = weights_results_sdf.withColumns(
    {
        "configuration_name": F.lit(CONFIGURATION_NAME),
        "variable_name": F.lit(VARIABLE_NAME)
    }
)
coverage_weights_sdf.show(4)

In [ ]:
ev.set_active_catalog("remote")
ev.active_catalog

In [ ]:
%%time
# Could create the table first using a migration and specify partitioning?
ev.write.to_warehouse(
    source_data=coverage_weights_sdf,
    table_name=TABLE_NAME,
    write_mode="append"
)

BUT! I already have it calculated locally

In [ ]:
weights_sdf = ev.table(table_name="nwm30_usgs_hires_basins_fractional_coverage").to_sdf()
weights_sdf.show(3)

In [ ]:
coverage_weights_sdf = weights_sdf.withColumnsRenamed(
    {
        "id": "location_id",
        "pos": "position_index"
    }
).withColumns(
    {
        "configuration_name": F.lit(CONFIGURATION_NAME),
        "variable_name": F.lit(VARIABLE_NAME)
    }
)
coverage_weights_sdf.show(4)

In [ ]:
ev.set_active_catalog("remote")
ev.active_catalog

In [ ]:
%%time
# Could create the table first using a migration and specify partitioning?
ev.write.to_warehouse(
    source_data=coverage_weights_sdf,
    table_name=TABLE_NAME,
    write_mode="create_or_replace",
    # uniqueness_fields=[
    #     "configuration_name",
    #     "variable_name",
    #     "position_index",
    #     "location_id"
    # ]
)

In [ ]:
ev.list_tables()

In [ ]:
grid_table = ev.table(table_name="grid_pixel_coverage_weights").to_sdf()
grid_table.show(4)